In [ ]:
# Importar as bibliotecas necessárias
import pandas as pd
import numpy as np
import os
import gc
from datetime import timedelta
from IPython.display import display

# Função para tocar som ao final (substitui `beep` do R)
def beep():
    print("\a")

# Carregar o primeiro dataframe
df = pd.read_csv("E:/Dropbox/transito/microdados/2022/out_dez_2022.csv")

# Converter as colunas de datas para o formato datetime
df['datetime_autuacao'] = pd.to_datetime(df['datetime_autuacao'], format='%Y-%m-%d')
df['data_pagamento'] = pd.to_datetime(df['data_pagamento'], format='%Y-%m-%d')

# Criar uma nova coluna 'paid' que indica se a multa foi paga
df['paid'] = df['valor_pago'] > 0

# Calcular o número de dias para pagamento, se pago
df['days_to_pay'] = (df['data_pagamento'] - df['datetime_autuacao']).dt.days

# Calcular a média dos dias para pagamento condicionada a múltiplas características onde 'paid' é True
media_dias_por_caracteristicas = df[df['paid'] & (df['days_to_pay'] > 0)].groupby(
    ['status_infracao', 'pontuacao', 'especie_veiculo']
).agg(
    mean_days_to_pay=('days_to_pay', 'mean'),
    count=('days_to_pay', 'size')
).reset_index()

# Exibir o resultado
display(media_dias_por_caracteristicas)

beep()

# Liberar memória
del df
gc.collect()

# Carregar nova base de dados para ajustar datas de pagamento de 2020 (primeiro semestre)
nova_base = pd.read_csv("E:/Dropbox/transito/microdados/2020/2020-01-01_2020-06-30.csv")
nova_base['datetime_autuacao'] = pd.to_datetime(nova_base['datetime_autuacao'], format='%Y-%m-%d')
nova_base['data_pagamento'] = pd.to_datetime(nova_base['data_pagamento'], format='%Y-%m-%d')

# Fazer merge com a média dos dias para pagamento
nova_base = nova_base.merge(media_dias_por_caracteristicas, on=['status_infracao', 'pontuacao', 'especie_veiculo'], how='left')
nova_base['data_estimada_pagamento'] = nova_base['datetime_autuacao'] + pd.to_timedelta(nova_base['mean_days_to_pay'].round(), unit='D')
nova_base['data_est_pag'] = np.where(
    nova_base['data_pagamento'].isna(),
    pd.NaT,
    np.where(nova_base['data_estimada_pagamento'] < nova_base['data_pagamento'], nova_base['data_estimada_pagamento'], nova_base['data_pagamento'])
)

# Salvar o resultado
nova_base.to_csv('E:/Dropbox/transito/microdados/2020/2020_1.csv', index=False)
beep()

# Liberar memória
del nova_base
gc.collect()

# Carregar e processar o segundo semestre de 2020
nova_base = pd.read_csv("E:/Dropbox/transito/microdados/2020/2020-07-01_2020-12-31.csv")
nova_base['datetime_autuacao'] = pd.to_datetime(nova_base['datetime_autuacao'], format='%Y-%m-%d')
nova_base['data_pagamento'] = pd.to_datetime(nova_base['data_pagamento'], format='%Y-%m-%d')

# Repetir o processo de merge e cálculo de datas estimadas de pagamento
nova_base = nova_base.merge(media_dias_por_caracteristicas, on=['status_infracao', 'pontuacao', 'especie_veiculo'], how='left')
nova_base['data_estimada_pagamento'] = nova_base['datetime_autuacao'] + pd.to_timedelta(nova_base['mean_days_to_pay'].round(), unit='D')
nova_base['data_est_pag'] = np.where(
    nova_base['data_pagamento'].isna(),
    pd.NaT,
    np.where(nova_base['data_estimada_pagamento'] < nova_base['data_pagamento'], nova_base['data_estimada_pagamento'], nova_base['data_pagamento'])
)

# Salvar o resultado
nova_base.to_csv('E:/Dropbox/transito/microdados/2020/2020_2.csv', index=False)
beep()

# Limpar memória
del nova_base
gc.collect()

# Caminho para a pasta onde estão os arquivos CSV de 2021
pasta = "E:/Dropbox/transito/microdados/2021"

# Listar todos os arquivos na pasta que terminam com '.csv'
arquivos_csv = [os.path.join(pasta, f) for f in os.listdir(pasta) if f.endswith('.csv')]

# Função para processar cada arquivo
def processar_arquivo(arquivo):
    nova_base = pd.read_csv(arquivo)
    nova_base['datetime_autuacao'] = pd.to_datetime(nova_base['datetime_autuacao'], format='%Y-%m-%d')
    nova_base['data_pagamento'] = pd.to_datetime(nova_base['data_pagamento'], format='%Y-%m-%d')

    # Merge e cálculo de datas estimadas de pagamento
    nova_base = nova_base.merge(media_dias_por_caracteristicas, on=['status_infracao', 'pontuacao', 'especie_veiculo'], how='left')
    nova_base['data_estimada_pagamento'] = nova_base['datetime_autuacao'] + pd.to_timedelta(nova_base['mean_days_to_pay'].round(), unit='D')
    nova_base['data_est_pag'] = np.where(
        nova_base['data_pagamento'].isna(),
        pd.NaT,
        np.where(nova_base['data_estimada_pagamento'] < nova_base['data_pagamento'], nova_base['data_estimada_pagamento'], nova_base['data_pagamento'])
    )

    # Salvar o arquivo processado
    nome_arquivo_saida = os.path.join(os.path.dirname(arquivo), os.path.basename(arquivo).replace('.csv', '_processed.csv'))
    nova_base.to_csv(nome_arquivo_saida, index=False)

    # Mensagem de confirmação
    print(f"Arquivo processado: {nome_arquivo_saida}")

    # Limpeza de memória
    del nova_base
    gc.collect()

# Processar todos os arquivos na pasta
for arquivo in arquivos_csv:
    processar_arquivo(arquivo)

beep()
